## 유튜브 동영상에서 음원 추출

In [ ]:
# 유튜브 다운로드 라이브러리 설치
!pip install pytube

In [ ]:
from pathlib import Path
import os
from pytube import YouTube

link = 'https://www.youtube.com/watch?v=B_K6z3HiRAs'


# 주어진 링크를 사용하여 Youtue 객체 생성
y = YouTube(link)

# 스트림 중 (음성이 포함된)가장 높은 해상도를 선택하여 다운로드
mp4 = y.streams.get_highest_resolution().download()

# 다운로드 파일 체크
print(mp4)
!ls .

In [ ]:
# Jupyter Notebook 용 display 모듈 임포트
import IPython.display as ipd

In [ ]:
# 다운로드 받은 비디오  살펴보기
ipd.Video(Path(mp4).name, width=640)

In [ ]:
# 동영상 편집 라이브러리 moviepy 설치 (기존 수업 자료에서 이미 설치되었을 것임)
#!pip install moviepy

In [ ]:
# moviepy 의 editor, audio.AudioCliop 모듈 임포트
import moviepy.editor as mpe
import moviepy.audio.AudioClip as mpa

# 오디오 클립 읽기
ac = mpe.AudioFileClip(mp4)

# 넘파이 배열로 변환
raw = ac.to_soundarray()

# shape 출력
print(raw.shape)

# min, max, mean, std 출력
print(f'min:{raw.min():.2f}, max:{raw.max():.2f}, mean:{raw.mean():.2f}, std:{raw.std():.2f}')

# sampling rate,  음성 길이 출력
print('fps:', ac.fps, 'seconds:', raw.shape[0]//ac.fps, ', duration:', ac.duration)

# 모노 사운드로 변환
raw_m = raw.mean(axis=1)

In [ ]:
# 넘파이 타입의 모노 사운드 재생 
ipd.Audio(raw_m, rate=ac.fps)

## 복습 : 사운드 시각화

In [ ]:
# 사운드 처리 패키지 librosa 설치 (기존 수업 자료에서 설치되었을 것임)
#!pip install librosa

In [ ]:
# librosa display 모듈 임포트
import librosa.display

# 넘파이 배열을 사운드 처리 정보를 추가하여 그려 보기
librosa.display.waveplot(raw_m, sr=ac.fps, alpha=0.5)

In [ ]:
# 데이터 시각화 라이브러리 plotly 설치 (기존 수업 자료에서 설치되었을 것임)
#!pip install plotly

In [ ]:
# plotly express 로 그려보기 
import plotly.express as px
sub = raw_m[60*ac.fps: 70*ac.fps]
print(sub.shape)
px.line(y=sub)

In [ ]:
# resample
sr = 22050
sub = librosa.resample(sub, ac.fps, sr)
print(sub.shape)
px.line(y=sub)

---
### 소리 듣기 이해

In [ ]:
from PIL import Image
Image.open('images/0_ear.jpg') # 그림 출처 : http://www.kmooc.kr/courses/course-v1:SNUk+SNU059.019k+2021_S2/course/

---
## 소리 데이터 전처리
* 1차원 소리 데이터터를 2차원 그림 데이터로 변환

In [ ]:
Image.open('images/1_spec.jpg') # 그림 출처: https://www.youtube.com/watch?v=Bk8phnnOFdg

In [ ]:
Image.open('images/2_spec.jpg') # 그림 출처: https://www.youtube.com/watch?v=Bk8phnnOFdg

In [ ]:
#Short-Time Fourier Transform 추출
FRAME_SIZE = 2048 
HOP_SIZE = 512
S_scale = librosa.stft(sub, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
sub.shape, S_scale.shape, sub.shape[0]/HOP_SIZE, S_scale[0][0]

In [ ]:
# 절대값 제곱 스펙트럼 계산
import numpy as np
S_power = np.abs(S_scale) ** 2
print(S_power.shape)
S_power.min(), S_power.max(), S_power.mean(), S_power.std()

In [ ]:
# 스펙트럼 그려보기
import matplotlib
import matplotlib.pyplot as plt
# 좀 더 선명하게 그리기 위한 jupyter magic을 지정한다.
%config InlineBackend.figure_format = 'retina'

def show_spec(value, sr):
    librosa.display.specshow(value, 
                            sr=sr, 
                            hop_length=HOP_SIZE, 
                            x_axis="time", 
                            y_axis="log",
                            cmap=matplotlib.cm.jet
                           )
    plt.clim(-100, 100)
    plt.colorbar(format="%+2.f")
show_spec(S_power, sr)

In [ ]:
# 데시벨로 변환
S_mel = 10*np.log10(S_power) 
S_mel.shape

In [ ]:
# 멜스펙트로그램 그려보기
S_mel = librosa.power_to_db(S_power)
show_spec(S_mel, sr)

---
## torchaudio로 멜스펙트로그램 만들기

In [ ]:
# pytorch 버전 업그레이드
!pip3 install torch==1.9.1+cpu torchvision==0.10.1+cpu torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# 버전 확인
import torch
import torchaudio
torch.__version__, torchaudio.__version__

In [ ]:
#  사전 준비:  wav 파일로 저장
!pip install soundfile
import soundfile as sf
sf.write('sub.wav', raw[60*ac.fps: 70*ac.fps], ac.fps)
ipd.Audio('sub.wav')

In [ ]:
# torchaudio를 이용하여 로드
signal, sr = torchaudio.load('sub.wav')
signal.shape, sr

In [ ]:
# sampling rate 조정
def resample(signal, cur_sr, target_sr): 
    resampler = torchaudio.transforms.Resample(cur_sr, target_sr)
    signal = resampler(signal)
    return signal, target_sr
signal, sr = resample(signal, sr, sr//2)
signal.shape, sr

In [ ]:
# 스테레오를 모노로 변환
def mix_down(signal):
    signal = torch.mean(signal, dim=0, keepdim=True)
    return signal
signal = mix_down(signal)
signal.shape

In [ ]:
# 멜스펙트로그램 변환 함수 생성
mel_spectrogram_trsf = torchaudio.transforms.MelSpectrogram(
        sample_rate=sr,
        n_fft=2048,
        hop_length=512,
        norm='slaney',
        n_mels=64
    )

In [ ]:
#변환
t_mel = mel_spectrogram_trsf(signal)
t_mel.shape

In [ ]:
# 그려보기 위해 numpy 로 변환
n_mel = t_mel.squeeze().cpu().numpy()
show_spec(n_mel, sr)

In [ ]:
!rm -rf wav_jpgs

In [ ]:
# 데시벨로 변환
n_mel = 10*np.log10(n_mel)
show_spec(n_mel, sr)